In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
# 데이터 로드
train_data = pd.read_excel('train_data.xlsx')
train_data = train_data.drop(['SP','결과값(연체회차)'], axis=1)
train_data = pd.get_dummies(train_data, columns=['AD_NO', 'LIV_ADD', 'RES_ADD', 'ADD_YN'])

In [3]:
# 입력 (X) / 출력 (Y) 변수 분리
X = train_data.drop('CB', axis=1)
y = train_data['CB']

In [4]:
# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# 특성 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Linear Regression

In [6]:
# 모델 학습
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [7]:
# 예측
train_pred = lr.predict(X_train)
test_pred = lr.predict(X_test)

In [8]:
# MSE 계산
train_mse = mean_squared_error(y_train, train_pred)
test_mse = mean_squared_error(y_test, test_pred)
print(f'Train MSE: {train_mse:.4f}')
print(f'Test MSE: {test_mse:.4f}')

Train MSE: 3.1564
Test MSE: 3.1276


In [9]:
# 피처 중요도 확인
coefficients = lr.coef_
features = X.columns.tolist()

In [10]:
# 피처 중요도를 데이터프레임으로 변환
feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': coefficients})

In [11]:
# 중요도를 백분율로 변환
total_importance = sum(abs(coefficients))
lr_feature_importance_percentages = (abs(coefficients) / total_importance) * 100
lr_feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': lr_feature_importance_percentages})
lr_feature_importance_df = lr_feature_importance_df.sort_values(by='Importance', ascending=False)
lr_feature_importance_df.to_excel('CB_LinearRegression_feature_importance.xlsx', index=False)

### Test_data.xlsx CB예측

In [12]:
# test.xlsx 파일 로드
test_data = pd.read_excel('test_data.xlsx')

In [13]:
# test_data에 동일한 전처리 수행
test_data = test_data.drop(['SP','결과값(연체회차)'], axis=1)
test_data = pd.get_dummies(test_data, columns=['AD_NO', 'LIV_ADD', 'RES_ADD', 'ADD_YN'])

In [14]:
# 훈련 데이터와 동일한 특성을 가지도록 조정
missing_cols = set(X.columns) - set(test_data.columns)
for c in missing_cols:
    test_data[c] = 0
test_data = test_data[X.columns]

In [15]:
# 스케일링
test_data_scaled = scaler.transform(test_data)

In [16]:
# 훈련된 모델을 사용하여 예측 수행
test_pred = lr.predict(test_data_scaled)

In [19]:
# 예측 결과를 데이터프레임으로 변환
test_pred_df = pd.DataFrame(test_pred.astype(int), columns=['CB 예측'])

In [20]:
# 결과를 엑셀 파일로 저장
test_pred_df.to_excel('test_LinearRegression_CB.xlsx', index=False)